In [3]:
pip install gradio

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/46.5 MB ? eta -:--:--
   ------ --------------------------------- 7.3/46.5 MB 39.9 MB/s eta 0:00:01
   ----------- ---------------------------- 12.8/46.5 MB 31.7 MB/s eta 0:00:02
   ------------ --------------------------- 14.9/46.5 MB 24.2 MB/s eta 0:00:02
   ----------------- ---------------------- 19.9/46.5 MB 24.4 MB/s eta 0:00:02
   --------------------- ------------------ 25.2/46.5 MB 24.6 MB/s eta 0:00:01
   ----------------------- ---------------- 27.0/46.5 MB 22.3 MB/s eta 0:00:01
   ---------------------------- ----------- 33.0/46.5 MB 23.1 MB/s eta 0:00:01
   ------------------------------- -------- 36.7/46.5 MB 23.8 MB/s eta 0:00:01
   -------------------------------- ------- 37.7/46.5 MB 20.6 MB/s eta 0:00:01
   ------------------------------------- -- 43.5/46.5 MB 21.3 MB/s eta 0:00:01
   ---------------------------------------- 46.5/46.5 MB 20.9 M

In [ ]:
import os  
import requests
import json
import base64
from openai import AzureOpenAI  
from dotenv import load_dotenv

def request_gpt(user_text):
    # .env 파일 로드
    load_dotenv()
    
    # 환경 변수에서 값 불러오기
    endpoint = os.getenv("ENDPOINT_URL", "https://team7-2.openai.azure.com/openai/deployments/teaam7-gpt4o/extensions/chat/completions?api-version=2024-02-15-preview")  
    deployment = os.getenv("DEPLOYMENT_NAME", "teaam7-gpt4o")  
    search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://6a021-aisearch.search.windows.net")  
    search_key = os.getenv("SEARCH_KEY", "search_key")  
    search_index = os.getenv("SEARCH_INDEX_NAME", "housing")  
    subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "subscription_key")
    
    # API 키와 엔드포인트 검증
    if not endpoint or not subscription_key:
        return "API endpoint 또는 API key가 설정되지 않았습니다."
    
    try:
        # Azure OpenAI 클라이언트 초기화
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=subscription_key,
            api_version="2024-05-01-preview",
        )
        
        # 프롬프트 구성
        chat_prompt = [
            {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "너는 부동산 및 청년 주택 매매·임대 관련 공문, 정책, 사업 안내서를 기반으로 사용자의 질문에 가장 정확하고 실용적인 정보를 제공하는 AI 어시스턴트야.\n\n현재 수많은 문서들이 벡터 DB에 저장되어 있으며, 너는 사용자의 질문에 대해 **가능한 한 한 개의 문서**에서만 정보를 추출하여 응답해야 해.\n\n### 응답 지침:\n\n1. 반드시 **사용자 질문과 가장 관련 있는 단일 문서 하나**를 선택하여, 그 문서 내용만 기반으로 응답해라.\n2. 질문이 **날짜(연도, 월 등)** 관련된 경우에는, 가능한 한 **정확한 날짜 정보**를 포함하여 응답하라. (예: \"2024년 3월 기준\", \"2023년 말 발표\" 등)\n3. 문서 내 **표(table) 형식의 데이터가 존재할 경우**, 질문과 관련된 정보를 표에서 우선적으로 추출하여 간결하고 보기 좋은 형식으로 정리해서 제시하라.\n4. 문서에서 인용 시에는 반드시 다음 정보를 포함하라:\n   - 문서 제목\n   - 발표일 또는 기준일\n   - 출처 기관(있는 경우)\n5. 문서에 해당 정보가 **명시되지 않았을 경우**, 허위로 유추하지 말고 \"문서에 명시되어 있지 않습니다\"라고 명확히 응답하라.\n6. 복수 문서를 참조해야만 가능한 질문의 경우에는, 먼저 단일 문서에서 최대한 응답한 뒤, \"기타 문서에서도 추가 정보가 있을 수 있습니다\"와 같이 부연하라.\n7. 문서와 관련 없는 일반적인 해석이나 조언은 하지 말고, 반드시 문서 근거 기반으로만 응답하라.\n\n### 예시 응답 포맷:\n\n질문: 2024년 청년 전세임대 지원 금액은?\n\n응답:\n> 국토교통부가 발표한 「2024년 청년 전세임대 사업안내」에 따르면 (2024.02 기준):\n>\n> | 지역 구분 | 최대 지원 한도 |\n> |-----------|----------------|\n> | 수도권   | 1.2억 원       |\n> | 광역시   | 9천만 원       |\n> | 기타 지역| 8천만 원       |\n>\n> ※ 출처: 국토교통부, 「2024 청년 전세임대 사업안내」, 2024년 2월\n\n"
                }
            ]
            }
        ]
        
        # 메시지 구성
        messages = chat_prompt  
        
        # 데이터 소스 구성
        data_sources = None
        if search_endpoint and search_key and search_index:
            data_sources = [{
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": search_index,
                    "semantic_configuration": "default",
                    "query_type": "vector_semantic_hybrid",
                    "fields_mapping": {},
                    "in_scope": True,
                    "role_information": "",
                    "filter": None,
                    "strictness": 4,
                    "top_n_documents": 5,
                    "authentication": {
                        "type": "api_key",
                        "key": search_key
                    },
                    "embedding_dependency": {
                        "type": "integrated"
                    }
                }
            }]
        
        # 응답 생성 요청
        completion_params = {
            "model": deployment,
            "messages": messages,
            "max_tokens": 800,  
            "temperature": 0.7,  
            "top_p": 0.95,  
            "frequency_penalty": 0,  
            "presence_penalty": 0,
            "stop": None,  
            "stream": False,
        }
        
        # 데이터 소스가 있을 경우에만 추가
        if data_sources:
            completion_params["extra_body"] = {"data_sources": data_sources}
        
        completion = client.chat.completions.create(**completion_params)
        
        # 응답 추출 및 반환
        if hasattr(completion, 'choices') and len(completion.choices) > 0:
            response_content = completion.choices[0].message.content
            return response_content
        else:
            return json.dumps(completion.model_dump(), ensure_ascii=False)
            
    except Exception as e:
        return f"오류 발생: {str(e)}"

def main():
    print("Azure OpenAI 부동산 상담 AI (종료하려면 'exit' 또는 '나가기' 입력)")
    print("-----------------------------------------------------")
    
    while True:
        user_input = input("질문: ")
        if user_input.lower() in ['exit', '나가기']:
            print("프로그램을 종료합니다.")
            break
            
        print("\n처리 중...\n")
        response = request_gpt(user_input)
        print(f'질문: {user_input}')
        print('-' * 50)
        print(f'답변: {response}')
        print('*' * 50)
        print()

if __name__ == "__main__":
    main()

Azure OpenAI 부동산 상담 AI (종료하려면 'exit' 또는 '나가기' 입력)
-----------------------------------------------------

처리 중...

질문: 안녕
--------------------------------------------------
답변: 오류 발생: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}
**************************************************

프로그램을 종료합니다.
